In [ ]:
!pip install pandas

In [ ]:
#importing the csv file into pandas data frame (had prob cuz of directory, directly pasted on same directory as this notebook)
#we see below that the data frame is loaded with all the data loaded from the csv file
df1 = pd.read_csv("Bengaluru_House_Data.csv")
df1.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [ ]:
#to find directory of this notebook
import os
os.getcwd()

'C:\\Users\\stlp\\Untitled Folder'

In [ ]:
df1.shape

(13320, 9)

In [ ]:
#to see sample in each of the data type category
#groupby your data type and you aggregate the count
df1.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

In [ ]:
#reducing the features, availability is not that important, society, area type.
df2 = df1.drop(['area_type', 'society', 'balcony', 'availability'],axis = 'columns')
df2.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [ ]:
#data cleaning (hadnling the NA or null values), on data frame, if u run isnull and .sum -> will tell no of rows where particular column value is NA
df2.isnull().sum()
#73 values, where bathroom is not available

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [ ]:
#dropping above values, you can also fill the median of all bathroom values into the NA values. anything works, new dataframe df3
df3 = df2.dropna()
df3.isnull().sum()


location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [ ]:
df3.shape

(13246, 5)

In [ ]:
df3['size'].unique() #4 bedroom and 4 bhk are same

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [ ]:
#creating a new column bhk, which is created based on the size column -> apply some func on size col
#you can use the string (2 bhk) and tokenize using the spaceand take first token 2 which will be your size value
#use python lamda function, x will contain column value for each of the rows one by one, and on those values, you
#want to apply some tranformation: if x = 2 bhk, split the string using space; split -> gives two tokens, out of which take first token
#it is still string, convert to a int
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

C:\Users\stlp\AppData\Local\Temp\ipykernel_5808\2045615459.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


In [ ]:
df3.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [ ]:
df3['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [ ]:
#what? 43 bedroom, lets see what that is
df3[df3.bhk>20]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [ ]:
#above looks like an error, 2400 sqft 43? to tackle this error, need to focus on this total_sqft as well
df3.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [ ]:
#a range is there above, need to calculate single number.  it should be float
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
#take range as an input, return the average place
def convert_sqft_to_num(x):
    tokens = x.split('-') #splitting here
    if len(tokens) == 2:   #if length is 2, we convert indiv to float no, and take average, if normal number, convert into float
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
#going to try line, example
convert_sqft_to_num('2166')

2166.0

In [ ]:
convert_sqft_to_num('2100 - 2850') #average

2475.0

In [ ]:
convert_sqft_to_num('34sq. meter') #not returning anything

In [ ]:
#applying this function and creating new dataframe df4
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)
df4.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2


In [ ]:
df4.loc[30] #data frame looks betterrr

location      Yelahanka
size              4 BHK
total_sqft       2475.0
bath                4.0
price             186.0
bhk                   4
Name: 30, dtype: object

# 3) Feature Engineering

In [ ]:
#creating feature that will be helpful for outlier detection and removal
#we have price per sqft column here
df5 = df4.copy()
df5['price_per_sqft']= df5['price']*100000/df5['total_sqft']
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [ ]:
#lets explore location column, how many rows for this?
#number of unique locations
df5.location.unique()

array(['Electronic City Phase II', 'Chikka Tirupathi', 'Uttarahalli', ...,
       '12th cross srinivas nagar banshankari 3rd stage',
       'Havanur extension', 'Abshot Layout'], dtype=object)

In [ ]:
len(df5.location.unique())
# 1300 is too much columns, we can convert few into dummies using hot encoding -> dimentionality curse

1304

In [ ]:
#other category? few locations only three to four data points,
# stripping any space from the data using lambda function
df5.location = df5.location.apply(lambda x: x.strip()) # spaces removed
#statistics of locations, this will give the location column, where we can perform aggregation function, count per location
#sorting by data points
location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

location
Whitefield               535
Sarjapur  Road           392
Electronic City          304
Kanakpura Road           266
Thanisandra              236
                        ... 
1 Giri Nagar               1
Kanakapura Road,           1
Kanakapura main  Road      1
Karnataka Shabarimala      1
whitefiled                 1
Name: location, Length: 1293, dtype: int64

In [ ]:
len(location_stats[location_stats<=10])

1052

In [ ]:
#data points
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

location
Basapura                 10
1st Block Koramangala    10
Gunjur Palya             10
Kalkere                  10
Sector 1 HSR Layout      10
                         ..
1 Giri Nagar              1
Kanakapura Road,          1
Kanakapura main  Road     1
Karnataka Shabarimala     1
whitefiled                1
Name: location, Length: 1052, dtype: int64

In [ ]:
#categorizing to other
#it's df5 = location column
len(df5.location.unique()) #after xstrip

1293

In [ ]:
#transformation
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())
#this will be good. just 242 can be used for hot encoding

242

In [ ]:
df5.head(10)

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,3247.863248
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,7467.057101
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,18181.818182
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4828.244275
9,other,6 Bedroom,1020.0,6.0,370.00,6,36274.509804


# 4) Outlier Removal

In [ ]:
#lets see how many rows in the data frame. we need to remove the anomalies/ data errors.
df5.shape

(13246, 7)

In [ ]:
#lets create a new data frame df6
#remove the above outliers, by just negating it and giving it to df6
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape #removed all the outliers

(12502, 7)

In [ ]:
#we can check for more outliers, price per sqft either very very high or very very low
#describe method gives some statistics on a particular column
df6.price_per_sqft.describe()
#267 is very un;ikely in bangalore
#max is also very very high, maybe possible, but while building a model -> good to remove these generic models

count     12456.000000
mean       6308.502826
std        4168.127339
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

In [ ]:
#we can create a function that removes these extreme cases using standard deviations
#assuming our data set has a normal distribution -> most of 68% data points should lie between mean and one standard deviation
#filtering out anything that is beyond one standard deviation
#we need to do it per location -> it might vary per location
def remove_pps_outliers(df):  #taking data frame as an input
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'): #grouping df by location, per location, getting subdata frame: for which
        m = np.mean(subdf.price_per_sqft) #calculating mean and std
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft> (m-st)) & (subdf.price_per_sqft<=(m+st))] #filtering and keeping this in reduced df
        df_out = pd.concat([df_out, reduced_df], ignore_index = True) #and keep on appending those dataframes per location
    return df_out #it will give us the output data frame.

#lets call this df7, if i call outliers on df6 -> we get df7
df7 = remove_pps_outliers(df6) #removing per per sqft from df6
df7.shape

#from 12k we removed close to 2000 outliers from dataset

(10241, 7)

In [ ]:
# some of 3bhk with same spft has less price than 2 bhk, maybe amenities? lets visualize it
#rcparams library from matplotlib
def plot_scatter_chart(df,location): #this function takes location as well as df as input
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcparams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft, bhk2.price_per_sqft, color = 'blue', label = ' 2 BHK', s=50)
    plt.scatter(bhk3.total_sqft, bhk3.price_per_sqft, marker='+', color = 'green', label = ' 3 BHK', s=50)
    plt.xlabel("Total square feet area")
    plt.ylabel("Price per square feet")
    plt.title(location)
    plt.legend()

plot_scatter_chart(df7,"Rajaji Nagar")
#it shows AttributeError: module 'matplotlib' has no attribute 'rcparams'
(((((((((((((((((((((((((SOLVE THIS )))))))))))))))))))))))))

NameError: name 'matplotlib' is not defined

In [ ]:
import matplotlib
print(matplotlib.__version__)

In [ ]:
pip install matplotlib --upgrade

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\stlp\\anaconda3\\Lib\\site-packages\\matplotlib\\_c_internal_utils.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Using cached matplotlib-3.6.2-cp39-cp39-win_amd64.whl (7.2 MB)


In [ ]:
#we will see from the graph, that 3bhk < 2bhk same location also.
#so for a given location; we will build a dictionary of stats per bhk
#'1' : mean, std, count, like that for each bhk

def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'): #going through every locationin the frame
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'): #so for every df, creating a new df based on bhk
            bhk_stats[bhk] = {    #for bhk df, creating mean, st and count
                'mean': np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            } #once this for loop is over, we will run the same for loop again trying to exclude the data points whose value is less than mean of previous bhk
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices, axis = 'index')

#we will give after the removal to df8;
df8 = remove_bhk_outliers(df7)
df8.shape

#removed

(7329, 7)

In [ ]:
#giving same scatter plot to df8; majority gone
plot_scatter_chart(df8,"Rajaji Nagar")
(((((((((((((((((((((((((((((((((((plot this after solving prev error)))))))))))))))))))))))))))))))))))

In [ ]:
# now that outliers are removed, lets seeby plotting histogram
import matplotlib
matplotlib.rcparams["figure.figsize"] = (20,10)
plt.hist(df8.price_per_sqft, rwidth=0.8)
plt.xlabel("price per square feet")
plt.ylabel("count")
((((((((((((((((((((((((((((solve))))))))))))))))))))))))))))

#majority of data pts from 0 to 10000, like a normal distribution: looks good, lets see bathroom feature

AttributeError: module 'matplotlib' has no attribute 'rcparams'

In [ ]:
df8[df8.bath>10]
#need to remove any bathroom specific outliers?
#2 bhk and 4 bathroom? unusual. remove bathroom = bed + 2

,location,size,total_sqft,bath,price,bhk,price_per_sqft
5277,Neeladri Nagar,10 BHK,4000.0,12.0,160.0,10,4000.000000
8486,other,10 BHK,12000.0,12.0,525.0,10,4375.000000
8575,other,16 BHK,10000.0,16.0,550.0,16,5500.000000
9308,other,11 BHK,6000.0,12.0,150.0,11,2500.000000
9639,other,13 BHK,5425.0,13.0,275.0,13,5069.124424


In [ ]:
# bathroom feature, histogram by calling plt.hist
plt.hist(df8.bath, rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("count")


#we see that most of the properties are in 2 bathroom, very few outliers later, after meeting with the manager

NameError: name 'plt' is not defined

In [ ]:
df8[df8.bath>df8.bhk+2] #these are outliers

,location,size,total_sqft,bath,price,bhk,price_per_sqft
1626,Chikkabanavar,4 Bedroom,2460.0,7.0,80.0,4,3252.032520
5238,Nagasandra,4 Bedroom,7000.0,8.0,450.0,4,6428.571429
6711,Thanisandra,3 BHK,1806.0,6.0,116.0,3,6423.034330
8411,other,6 BHK,11338.0,9.0,1000.0,6,8819.897689


In [ ]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape    #now our df looks pretty much neat and clean
#now lets prepare for ML training. for that, need to remove unnecessary features
#price_per_sqft (only for outliers we made this) and size (we have bhk) = unnecessary

(7251, 7)

In [ ]:
#removing the above
df10 = df9.drop(['size', 'price_per_sqft'], axis = 'columns')

# 5) Model building

In [ ]:
#we cleaned up our data frame and got ready for model building
#we will build ml models -> use k-fold cross validation and grid cv to come up with the best algorithm and parameters.
#we know that location -> ml algorithms cannot read a text data.
#we need to convert location into a numeric column -> one hot encoding -> using pandas dummies method here.

#take your data frame, the column that you want to encode and call pd.getdummies function on it.

pd.get_dummies(df10.location)

#each of the location -> new columns
#this is pretty straightforward dummies (((((((((learn hot encoding)))))))))

#storing the dummy column into a separate data frame

dummies = pd.get_dummies(df10.location)
dummies.head(3)

#this contains just the numeric not the text
#concatenation with df10 and removing text in next code

,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,6th Phase JP Nagar,7th Phase JP Nagar,8th Phase JP Nagar,9th Phase JP Nagar,...,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df11 = pd.concat([df10,dummies.drop('other', axis = 'columns')], axis = 'columns')
df11.head(3)  #just to see a few

,location,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,1st Block Jayanagar,2850.0,4.0,428.0,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1st Block Jayanagar,1630.0,3.0,194.0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1st Block Jayanagar,1875.0,2.0,235.0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#we can remove the location col, now that dummies are added here
df12 = df11.drop('location', axis='columns')
df12.head(2)

,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,2850.0,4.0,428.0,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,194.0,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#our data processing pipeline has come till d12. 12th stage. let's start building the model

df12.shape

(7251, 245)

In [ ]:
# I want X to contain only independent variables/ drop the price column
X = df12.drop('price', axis = 'columns')
X.head()

,total_sqft,bath,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,6th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,2850.0,4.0,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1875.0,2.0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1200.0,2.0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1235.0,2.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y = df12.price
y.head()

0    428.0
1    194.0
2    235.0
3    130.0
4    148.0
Name: price, dtype: float64

In [ ]:
#we always divide our data set into training (model training ) and test dataset (evaluate performance)
#importing train test split
#20% of the samples to be test samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 10)

# (a) Linear regression

In [ ]:
#created a linear regression model
#math behinf linear is not used here.
#we are just going to call fit method on X train and Y train
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,Y_train) #model trained in this tab
lr_clf.score(X_test,y_test) #to evaluate the score of the model

0.8452277697874321

In [ ]:
#a data scientist would try couple of models, tuning the parameters etc
#so, now, thats why, we are going to use K-fold cross validation

#importing some methods
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

#creating a shuffle splitfor cross-validation (shufflesplit will randomize the sample, so that data samples are not targeted into one area)
cv = ShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)

cross_val_score(LinearRegression(), X, y, cv=cv)
#majority of the time 80%: even if we run 5 full cross validation, we are getting more than 80%)

array([0.82430186, 0.77166234, 0.85089567, 0.80837764, 0.83653286])

# (b) Grid Search cv


In [ ]:
#lets try other regression methods
#lets see which model gives us the best score.
# -> for that lets use a method called grid search cv
# grid search cv -> a good API that escalon provides that can run your model on different regressors and parameters & tell you the best score.

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

#for gridsearchcv, writing a function

def find_best_model_using_gridsearchcv(X,y): #finding the best model by supplying X, y as input
    algos = {                                          #have specified algorithm as well as the parameters
        'linear_regression' : {
            'model': LinearRegression(),   #grid search cv, will tell the best algo as well as params (hyperparameter tuning)
            'params': {
                'normalize': [True, False]   #this is the parameters grid that we will use
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random','cyclic']
            }
        },
        'decision_tree' : {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0) #cv shufflesplit which will just randomly shuffle the sample, to get more better result
    for algo_name, config in algos.items():             #here, just going through the python dictionary, algo is given above
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False) #& initializing gridsearchcv object with the model as well as params & for cv object we are using 5 cv
        gs.fit(X,y)  #calling the fit method
        scores.append({   #once the fit method is called, will append the scores into the scores list
            'model': algo_name,
            'best_score': gs.best_score_,   #these gs will tell the best score and best params for that model/particular run
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])  #to return the resultant scores into the data frame

#so once we have the above method defined. we can call the method on X & y

find_best_model_using_gridsearchcv(X,y)


#i got algos not defined. need to fix the indentation.

C:\Users\stlp\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\stlp\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = m

,model,best_score,best_params
0,linear_regression,0.818354,{'normalize': False}
1,lasso,0.687444,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.724695,"{'criterion': 'mse', 'splitter': 'best'}"


In [ ]:
X.columns #can put this separately in another cell and not with the below one, to get all the columns
np.where(X.columns=='2nd Phase Judicial Layout')[0][0] #understanding how the below loc_index method works internally.

5

In [ ]:
#The winner is linear regression with best params...
#we have already used lr clf classifier allocated before -> already trained with 84% score
#using the same thing to predict things out.

def predict_price(location,sqft,bath,bhk):   #so these functions will take these as input and output the estimated price
    loc_index = np.where(X.columns==location)[0][0]  #everything has a column, but for location, we have around 240 location columns.
                                                    #if you put location name in place of locations, u get the index
    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:        #if it is greater than or equal to 0; index will become 1
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

In [ ]:
predict_price('1st Phase JP Nagar', 1000, 2, 1)

C:\Users\stlp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


85.27192453606447

In [ ]:
predict_price('1st Phase JP Nagar', 1000, 3, 3)

C:\Users\stlp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


86.80519395208691

In [ ]:
predict_price('Indira Nagar', 1000, 2, 2)

C:\Users\stlp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


181.27815484006805

In [ ]:
predict_price('Indira Nagar', 1000, 2, 2)   #this is more costly than jp nagar

C:\Users\stlp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


181.27815484006805

In [ ]:
predict_price('1st Phase JP Nagar', 1000, 2, 3) #cheaper than 2,2: maybe on other data like sqft or we dont have much data points
#maybe we need more data.

C:\Users\stlp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


81.72616900757684

# (c) Exporting to a pickle file

In [ ]:
#now that our model is built; we will export this model to a pickle file.
#then it will be used by our python flask server
#so now we will export all the artifacts which are needed by our python flask server to different files

#     -> As a data scientist, worked with a business manager, went through various iterations, tried different models,
# grid search cv, full cross validation, cleaned the data, removed outliers, came up with a good model which is ready
# to be used for production.

#in real life, we will use a/b testing to test prev model vs new model

#we will see how to export this model and use it on our website.

import pickle  #importing a pickle file
with open('Bengaluru_House_Data_model.pickle','wb') as f:
    pickle.dump(lr_clf,f) # on you pass your model, classifier as an argument

In [ ]:
#to find directory of this notebook
import os
os.getcwd()
#pickle file is created -> size is just 9kb -> LR model is just storing the coefficients, intercepts etc.
#it doesnt have your actual data

'C:\\Users\\stlp\\Untitled Folder'

In [ ]:
#other than the model, we also need the column info -> X.column, their index into the list, structure is
#important for making our prediction.
#So will export those informations into a JSON file.

import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]  #converting the column into a lower case. it's better uniformity.
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))    #dumping all of them into JSON file.

#going to json file, editing with notepad. check -> it has all the columns.

#so this (json file) and (pickle model) is going to be used for python flask server.

#next -> going to build python flask server which will use these two artifacts.

# 6) Python Flask Server

In [ ]:
# Next step: to write a python flash server, which can sell HTTP requestsmade from the UI and it can predict the home prices for you.
# python flask server: back end for our UI application